In [1]:
import matplotlib.pyplot as plt
import pandas
import re
from pathlib import Path
import numpy as np
import random
import os

import Utilities

In [2]:
def SetCPList(cp_num):
    CPlist_NYC = Utilities.GetScenarioCSVList(cp_num, True, "NYC")
    CPlist_ITH = Utilities.GetScenarioCSVList(cp_num, True, "ITH")
    CPlist_ISR = Utilities.GetScenarioCSVList(cp_num, True, "ISR")
    return CPlist_NYC, CPlist_ITH, CPlist_ISR

In [15]:
def FirstOutAnalysis(CPlist, carA_out_direction, carB_out_direction, out_threshold):
    timeA_list = []
    timeB_list = []

    for cp in CPlist:
        # first truncate the scenario
        dfA, dfB = Utilities.GetTruncatedScenario(cp, out_threshold)

        # data doesn't have time column, so we need to create one
        dfA['Time'] = np.arange(len(dfA))  # arange returns evenly spaced values within a given interval
        dfB['Time'] = np.arange(len(dfB))

        timeA, timeB = 99999,99999

        if carA_out_direction == "top":
            # check if carA's Z position is ever greater than the threshold
            if dfA['HeadPosZA'].max() > out_threshold - 1:
                # if so, get the time when it first happens
                # create a new dataframe that only contains the rows where Z position is greater than threshold, then get the first row's time
                timeA = dfA.loc[dfA['HeadPosZA'] > out_threshold - 1, 'Time'].iloc[0]   
        elif carA_out_direction == "bottom":
            if dfA['HeadPosZA'].min() < -out_threshold + 1:
                timeA = dfA.loc[dfA['HeadPosZA'] < -out_threshold + 1, 'Time'].iloc[0]
        elif carA_out_direction == "left":
            if dfA['HeadPosXA'].min() < -out_threshold + 1:
                timeA = dfA.loc[dfA['HeadPosXA'] < -out_threshold + 1, 'Time'].iloc[0]
        elif carA_out_direction == "right":
            if dfA['HeadPosXA'].max() > out_threshold - 1:
                timeA = dfA.loc[dfA['HeadPosXA'] > out_threshold - 1, 'Time'].iloc[0]

        if carB_out_direction == "top":
            if dfB['HeadPosZB'].max() > out_threshold - 1:
                timeB = dfB.loc[dfB['HeadPosZB'] > out_threshold - 1, 'Time'].iloc[0]
        elif carB_out_direction == "bottom":
            if dfB['HeadPosZB'].min() < -out_threshold + 1:
                timeB = dfB.loc[dfB['HeadPosZB'] < -out_threshold + 1, 'Time'].iloc[0]
        elif carB_out_direction == "left":
            if dfB['HeadPosXB'].min() < -out_threshold + 1:
                timeB = dfB.loc[dfB['HeadPosXB'] < -out_threshold + 1, 'Time'].iloc[0]
        elif carB_out_direction == "right":
            if dfB['HeadPosXB'].max() > out_threshold - 1:
                timeB = dfB.loc[dfB['HeadPosXB'] > out_threshold - 1, 'Time'].iloc[0]

        timeA_list.append(timeA)
        timeB_list.append(timeB)
    
    print(timeA_list)
    print(timeB_list)

    out_first_list = []
    # compare which car is out first
    for i in range(len(timeA_list)):
        if timeA_list[i] < timeB_list[i]:
            out_first_list.append("A")
        elif timeA_list[i] > timeB_list[i]:
            out_first_list.append("B")
        else:
            out_first_list.append("same")
    print(out_first_list)
    # print the amount of A and B
    print(f"car A out first: {out_first_list.count('A')}, car B out first: {out_first_list.count('B')}")

    # print the percentage of A and B
    print(f"car A out first: {(out_first_list.count('A') / len(out_first_list))}, car B out first: {out_first_list.count('B') / len(out_first_list)}")

    # print call file path where a car is out first
    """
    for i in range(len(out_first_list)):
        if out_first_list[i] == "A":
            print(CPlist[i])
    """




In [21]:
# create a dictionary to store out direction of each cp
# key: cp number, value: out direction
cp_out_direction_A = {}
cp_out_direction_B = {}

cp_out_direction_A[1] = "left"
cp_out_direction_B[1] = "bottom"
cp_out_direction_A[2] = "left"
cp_out_direction_B[2] = "right"
cp_out_direction_A[3] = "left"
cp_out_direction_B[3] = "bottom"
cp_out_direction_A[5] = "left"
cp_out_direction_B[5] = "right"
cp_out_direction_A[6] = "right"
cp_out_direction_B[6] = "right"
cp_out_direction_A[7] = "top"
cp_out_direction_B[7] = "right"
cp_out_direction_A[8] = "left"
cp_out_direction_B[8] = "right"

my_cp = 1
CPlist_NYC, CPlist_ITH, CPlist_ISR = SetCPList(my_cp)
FirstOutAnalysis(CPlist_NYC, cp_out_direction_A[my_cp], cp_out_direction_B[my_cp], 20)

got 28 csv from CP1 in NYC with validity: True 
got 10 csv from CP1 in ITH with validity: True 
got 30 csv from CP1 in ISR with validity: True 
[344, 354, 99999, 99999, 99999, 323, 99999, 184, 99999, 178, 220, 259, 309, 272, 369, 407, 176, 240, 281, 99999, 897, 479, 337, 238, 207, 99999, 99999, 295]
[210, 99999, 159, 370, 234, 253, 206, 275, 170, 88, 333, 278, 164, 325, 337, 299, 299, 388, 201, 155, 99999, 638, 99999, 277, 278, 399, 469, 99999]
['B', 'A', 'B', 'B', 'B', 'B', 'B', 'A', 'B', 'B', 'A', 'A', 'B', 'A', 'B', 'B', 'A', 'A', 'B', 'B', 'A', 'A', 'A', 'A', 'A', 'B', 'B', 'A']
car A out first: 13, car B out first: 15
car A out first: 0.4642857142857143, car B out first: 0.5357142857142857
Data\CrossCultureResearchRawData\NYC11\csv\CSVScenario-CP1_Session-NYC11_2022-09-14-16-26-15.csv
Data\CrossCultureResearchRawData\NYC18\csv\CSVScenario-CP1_Session-NYC18_2022-09-19-11-41-57.csv
Data\CrossCultureResearchRawData\NYC20\csv\CSVScenario-CP1_Session-NYC20_2022-09-19-15-12-43.csv
Data\